<a href="https://colab.research.google.com/github/sandeepgundeboina/LearningSpark/blob/main/SparkDeltaMerge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install delta-spark==2.0.0

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("SparkDeltaMerge") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.0.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

In [2]:
from delta.tables import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
df=spark.read.format('csv').option('header','true').option('inferSchema',True).load('/content/drive/MyDrive/Abc/Ola_data/products.csv')

In [4]:
df.write.format('delta').mode('overwrite').saveAsTable('Products_table')

In [5]:
df.createOrReplaceTempView('Products_temp')

In [6]:
spark.sql('select * from products_temp').show(4)

+-----------------+---------------+----------+--------+-----+----------+-------+
|          product|          store|product_id|store_id|  MRP|     state|country|
+-----------------+---------------+----------+--------+-----+----------+-------+
|    Wired Earbuds|Electronics Hub|   PROD101| STORE01|25.99|California|    USA|
| Portable Charger|    Mobile Mart|   PROD102| STORE02| NULL|     Texas|    USA|
|Bluetooth Speaker|Electronics Hub|   PROD103| STORE01|79.50|California|    USA|
| Smart Light Bulb|   Home Gadgets|   PROD104| STORE03|15.00|   Florida|    USA|
+-----------------+---------------+----------+--------+-----+----------+-------+
only showing top 4 rows



In [7]:
spark.sql('create or replace table Products (product string,store string,product_id string,MRP int,\
            state string, country string)using delta location "/content/sample_data/store"')

DataFrame[]

In [8]:
spark.sql('select * from Products').show()

+-------------------+---------------+----------+----+----------+-------+
|            product|          store|product_id| MRP|     state|country|
+-------------------+---------------+----------+----+----------+-------+
|      Foot Massager|   Home Gadgets|   PROD175| 100|   Florida|    USA|
|       Dehumidifier|   Office Depot|   PROD162| 150|  Illinois|    USA|
|         Hand Mixer|   Office Depot|   PROD154|  25|  Illinois|    USA|
|        Smart Scale|    Mobile Mart|   PROD179|null|     Texas|    USA|
|       Portable SSD|      PC Palace|   PROD129|null|  New York|    USA|
|Digital Thermometer|   Home Gadgets|   PROD177|  15|   Florida|    USA|
|     Water Purifier|   Office Depot|   PROD158| 200|  Illinois|    USA|
|        Power Strip|   Home Gadgets|   PROD115|null|   Florida|    USA|
|         Laptop Bag|   Office Depot|   PROD140|  45|  Illinois|    USA|
|     Coffee Machine|   Home Gadgets|   PROD151|  95|   Florida|    USA|
|   Screen Protector|Electronics Hub|   PROD138|  1

In [9]:
spark.sql(
    "Merge into Products as target using Products_temp as source on target.product_id = source.product_id\
     when matched then update set target.product=source.product,\
        target.store=source.store,\
        target.product_id=source.product_id,\
        target.MRP=source.MRP,\
        target.state=source.state,\
        target.country=source.country\
     when not matched then insert * "
)

DataFrame[]

In [10]:
spark.sql('select * from Products').show(5)

+-----------------+---------------+----------+----+----------+-------+
|          product|          store|product_id| MRP|     state|country|
+-----------------+---------------+----------+----+----------+-------+
|    Wired Earbuds|Electronics Hub|   PROD101|  25|California|    USA|
| Portable Charger|    Mobile Mart|   PROD102|null|     Texas|    USA|
|Bluetooth Speaker|Electronics Hub|   PROD103|  79|California|    USA|
| Smart Light Bulb|   Home Gadgets|   PROD104|  15|   Florida|    USA|
|             NULL|    Mobile Mart|   PROD105|  49|     Texas|    USA|
+-----------------+---------------+----------+----+----------+-------+
only showing top 5 rows



**END OF CODE**